## Importación de Librerías

In [1]:
from google.colab import drive
import json
import polars as pl
import pandas as pd
import re
from collections import Counter

## Montar Google Drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# MANIPULACION DE ARCHIVOS REVIEWS

## Función para leer archivos JSOM

In [60]:
def leer_parquet(n, ubicacion):
    # Utiliza el prefijo /content/drive/MyDrive/ para acceder a los archivos
    file_path = f'/content/drive/MyDrive/reviews/{ubicacion}/{n}.parquet'

    # Lee el archivo Parquet en un DataFrame de Polars
    df = pl.read_parquet(file_path)

    return df

## Definir una lista de ubicaciones y nombres de carpetas

In [61]:
ubicaciones = ['review-Florida_parquet', 'review-New_York_parquet']
carpetas = {'review-Florida_parquet': 19, 'review-New_York_parquet': 18}

## Lectura y unificacion de archivos

In [62]:
# Lista para almacenar todos los DataFrames
dfs = []

for ubicacion in ubicaciones:
    # Recorrer todos los archivos en la carpeta actual
    for i in range(1, carpetas[ubicacion] + 1):

        # Leer el archivo parquet
        df = leer_parquet(str(i), ubicacion)

        # Agregar el DataFrame procesado a la lista
        dfs.append(df)

# Concatenar todos los DataFrames en uno solo
reviews_consolidado = pl.concat(dfs)

## Traslado de DataFrame a Pandas

In [105]:
# Primero, convertimos df1 a un DataFrame de pandas
df1 = reviews_consolidado .to_pandas()

In [106]:
df1.columns

Index(['user_id', 'time', 'rating', 'text', 'gmap_id'], dtype='object')

In [107]:
# Seleccionar solo las columnas necesarias
df1 = df1[['text', 'gmap_id']]

df1.columns

Index(['text', 'gmap_id'], dtype='object')

In [108]:
df1.head()

,text,gmap_id
0,Update: Their “reply” to my review amounted to...,0x8893863ea87bd5dd:0x9383ebf973e74abb
1,He's a knowledgeable doctor but the way he run...,0x8893863ea87bd5dd:0x9383ebf973e74abb
2,"Best doctor I've ever had, I never wait to be ...",0x8893863ea87bd5dd:0x9383ebf973e74abb
3,I was told he is a good doctor. I was trying t...,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,Takes the time to actually get to know his pat...,0x8893863ea87bd5dd:0x9383ebf973e74abb


In [109]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5550000 entries, 0 to 5549999
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   text     object
 1   gmap_id  object
dtypes: object(2)
memory usage: 84.7+ MB


## Gmap_id unicos

In [115]:
cantida = df1['gmap_id'].nunique()
cantida

129644

# MANIPULACION .CSV HOTELES

## Lectura de Hoteles.csv

In [69]:
ruta_hoteles='/content/drive/MyDrive/reviews/hoteles.csv'

# Lee el archivo CSV en un DataFrame de pandas
df2 = pd.read_csv(ruta_hoteles)

In [70]:
df2.head()

,name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,url,County,City,State,Country
0,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,28.510625,-81.418326,Hotel,2.8,18,https://www.google.com/maps/place//data=!4m2!3...,Orange County,Orlando,Florida,United States
1,Building 71,0x88dd8081a8007249:0xaebc4131dc396da9,28.402668,-81.468409,Hotel,4.8,25,https://www.google.com/maps/place//data=!4m2!3...,Orange County,Orlando,Florida,United States
2,Luxurious Apts in Brickell,0x88d9b68731cfdd8f:0xe4dcbb0342098ef,25.759244,-80.193571,Hotel,5.0,5,https://www.google.com/maps/place//data=!4m2!3...,Miami-Dade County,Miami,Florida,United States
3,Times Square Towers,0x89c258ffcaad98a9:0x2c9c3733efc6ef43,40.755883,-73.980077,Hotel,4.2,5,https://www.google.com/maps/place//data=!4m2!3...,New York County,New York,New York,United States
4,Gilt,0x89c258fc6c3037df:0x4c0fd32b72899779,40.757880,-73.975152,Hotel,4.4,8,https://www.google.com/maps/place//data=!4m2!3...,New York County,New York,New York,United States


## Eliminacion de Columnas innecesarias

In [71]:
# Seleccionar solo las columnas necesarias
df2 = df2[['name', 'gmap_id', 'City', 'State', 'url']]

df2.columns

Index(['name', 'gmap_id', 'City', 'State', 'url'], dtype='object')

In [72]:
df2.head()

,name,gmap_id,City,State,url
0,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...
1,Building 71,0x88dd8081a8007249:0xaebc4131dc396da9,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...
2,Luxurious Apts in Brickell,0x88d9b68731cfdd8f:0xe4dcbb0342098ef,Miami,Florida,https://www.google.com/maps/place//data=!4m2!3...
3,Times Square Towers,0x89c258ffcaad98a9:0x2c9c3733efc6ef43,New York,New York,https://www.google.com/maps/place//data=!4m2!3...
4,Gilt,0x89c258fc6c3037df:0x4c0fd32b72899779,New York,New York,https://www.google.com/maps/place//data=!4m2!3...


In [73]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     161 non-null    object
 1   gmap_id  161 non-null    object
 2   City     161 non-null    object
 3   State    161 non-null    object
 4   url      161 non-null    object
dtypes: object(5)
memory usage: 6.4+ KB


# REALIZACIÓN DE JOIN DE LOS DOS DATAFRAME

In [74]:
df = pd.merge(df2, df1, on='gmap_id', how='left')

In [75]:
df.head()

,name,gmap_id,City,State,url,text
0,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,I'm new to the area and needed a place to live...
1,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,This was previously a Ramada but was sold & be...
2,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,Just checked in and found out that it is under...
3,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,"I like the place. The place is massive, and su..."
4,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,Awesome staff place is nice and clean rooms ar...


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 880 entries, 0 to 879
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     880 non-null    object
 1   gmap_id  880 non-null    object
 2   City     880 non-null    object
 3   State    880 non-null    object
 4   url      880 non-null    object
 5   text     439 non-null    object
dtypes: object(6)
memory usage: 48.1+ KB


## Elimino las filas con valores nulos en text

In [77]:
df = df.dropna(subset=['text'])

## Creacion de Carpeta amenidades

In [78]:
df['amenidades'] = ''

In [79]:
cantidad_valores_unicos = df1['gmap_id'].nunique()
cantidad_valores_unicos

129644

## Funcion para extraccion de amenidades a partir de la columa text

In [80]:
def extraer_amenidades(df, palabras_a_buscar):
    """
    Función para extraer palabras clave de amenidades del texto y agregarlas en una lista en la columna 'amenidades'.

    Args:
    - df: DataFrame de pandas que contiene la columna 'text' donde buscar las palabras clave.
    - palabras_a_buscar: Lista de palabras clave que se buscarán en el texto.

    Returns:
    - None (el DataFrame se actualiza inplace).
    """
    # Iterar sobre cada fila del DataFrame
    for index, row in df.iterrows():
        texto = row['text']

        # Verificar si el texto no es None
        if texto is not None:
            # Inicializar una lista vacía para almacenar las amenidades encontradas
            amenidades_encontradas = []

            # Buscar palabras clave de amenidades en el texto y agregarlas a la lista de amenidades encontradas
            for palabra in palabras_a_buscar:
                if palabra in texto:
                    amenidades_encontradas.append(palabra)

            # Asignar la lista de amenidades encontradas a la columna 'amenidades' para esta fila
            df.at[index, 'amenidades'] = amenidades_encontradas
        else:
            # Si el texto es None, asignar una lista vacía a 'amenidades'
            df.at[index, 'amenidades'] = []

### Entrada de palabra claves para amenidades

In [81]:
palabras_a_buscar = ['wifi', 'gym', 'restaurant', 'piscina', 'laundry', 'kindergarten', 'pool', 'parking lot', 'massage', 'casino', 'resort', 'fitness', 'tennis', 'taxi', 'Swimming', 'food', 'parking', 'central heating', 'central heat']

### Llamada de la función para extraer las amenidades

In [82]:
extraer_amenidades(df, palabras_a_buscar)
df.head()

,name,gmap_id,City,State,url,text,amenidades
0,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,I'm new to the area and needed a place to live...,[pool]
1,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,This was previously a Ramada but was sold & be...,"[laundry, pool, parking lot, parking]"
2,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,Just checked in and found out that it is under...,[]
3,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,"I like the place. The place is massive, and su...",[]
4,Palms Inn,0x88e77bee8518ceef:0xbbd071e414788286,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,Awesome staff place is nice and clean rooms ar...,[]


    
    
## Función para crear un nuevo DataFrame con las amenidades unidas para gmap_id repetidos.

In [83]:
def crear_df_definitivo(df):
    """
    Args:
    - df: DataFrame de pandas que contiene las columnas 'gmap_id' y 'amenidades'.

    Returns:
    - DataFrame: Nuevo DataFrame con las amenidades unidas para gmap_id repetidos.
    """
    # Inicializar un diccionario para almacenar las amenidades para cada gmap_id
    gmap_id_amenidades = {}

    # Iterar sobre cada fila del DataFrame
    for index, row in df.iterrows():
        gmap_id = row['gmap_id']
        amenidades = row['amenidades']

        # Verificar si el gmap_id ya existe en el diccionario
        if gmap_id in gmap_id_amenidades:
            # Si el gmap_id ya existe, unir las amenidades con las existentes
            if amenidades:
                gmap_id_amenidades[gmap_id] = list(set(gmap_id_amenidades[gmap_id] + amenidades))
        else:
            # Si el gmap_id no existe, agregarlo al diccionario
            gmap_id_amenidades[gmap_id] = amenidades

    # Convertir el diccionario a un nuevo DataFrame
    df_definitivo = pd.DataFrame(list(gmap_id_amenidades.items()), columns=['gmap_id', 'amenidades'])

    # Fusionar con el DataFrame original para conservar todas las columnas
    df_definitivo = pd.merge(df_definitivo, df.drop(columns=['amenidades']).drop_duplicates(), on='gmap_id', how='left')

    return df_definitivo


## Llamar a la nueva función para crear el DataFrame definitivo

In [84]:
df_definitivo = crear_df_definitivo(df)
df_definitivo.head()

,gmap_id,amenidades,name,City,State,url,text
0,0x88e77bee8518ceef:0xbbd071e414788286,"[parking lot, laundry, parking, pool]",Palms Inn,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,I'm new to the area and needed a place to live...
1,0x88e77bee8518ceef:0xbbd071e414788286,"[parking lot, laundry, parking, pool]",Palms Inn,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,This was previously a Ramada but was sold & be...
2,0x88e77bee8518ceef:0xbbd071e414788286,"[parking lot, laundry, parking, pool]",Palms Inn,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,Just checked in and found out that it is under...
3,0x88e77bee8518ceef:0xbbd071e414788286,"[parking lot, laundry, parking, pool]",Palms Inn,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,"I like the place. The place is massive, and su..."
4,0x88e77bee8518ceef:0xbbd071e414788286,"[parking lot, laundry, parking, pool]",Palms Inn,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,Awesome staff place is nice and clean rooms ar...


## Funcion para eliminar filas repetidas

In [85]:
def eliminar_filas_repetidas(df):

    # Eliminar filas duplicadas basadas en el gmap_id
    df_sin_repetidos = df.drop_duplicates(subset=['gmap_id'])

    return df_sin_repetidos


In [86]:
# Llamar a la nueva función para eliminar filas repetidas
df_sin_repetidos = eliminar_filas_repetidas(df_definitivo)

In [87]:
df_sin_repetidos.reset_index(drop=True, inplace=True)

In [88]:
df_sin_repetidos.head()

,gmap_id,amenidades,name,City,State,url,text
0,0x88e77bee8518ceef:0xbbd071e414788286,"[parking lot, laundry, parking, pool]",Palms Inn,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,I'm new to the area and needed a place to live...
1,0x88dd80812203577d:0x5f77db1e909b56e9,[],Building 69,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,"Very clean, modern and fully loaded with all o..."
2,0x88e77dd4b14a224d:0x6aa10d93b20597ae,"[parking lot, parking]","The Florida Hotel & Conference Center, BW Prem...",Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,"Hotel was great, room was clean, access to the..."
3,0x88d9c7f1d1a779df:0xbc923669c3837bf4,"[parking, wifi, pool]",The Best Miami Hotel,Miami,Florida,https://www.google.com/maps/place//data=!4m2!3...,"The room was clean, and I had no problems chec..."
4,0x88e7648b437faec1:0xfaebd98e821b5831,"[food, parking]",Ventura Resort Rentals Orlando,Orlando,Florida,https://www.google.com/maps/place//data=!4m2!3...,"Easy parking, pleasant Staff.decent food ..lit..."


In [89]:
df_sin_repetidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gmap_id     27 non-null     object
 1   amenidades  27 non-null     object
 2   name        27 non-null     object
 3   City        27 non-null     object
 4   State       27 non-null     object
 5   url         27 non-null     object
 6   text        27 non-null     object
dtypes: object(7)
memory usage: 1.6+ KB


In [90]:
ruta_excel = '/content/drive/MyDrive/reviews/df_sin_repetidos.xlsx'

In [91]:
df_sin_repetidos.to_excel(ruta_excel, index=False)

pruebas

In [92]:
hoteles2='/content/drive/MyDrive/reviews/hotelesv2.parquet'

# Lee el archivo CSV en un DataFrame de pandas
car = pd.read_parquet(hoteles2)

In [93]:
car.head()

,name,latitude,longitude,County,City,State,Country
0,SOFITEL MIAMI,25.78307,-80.29356,Miami-Dade County,Miami,Florida,United States
1,HILTON MIAMI AIRPORT,25.78241,-80.28058,Miami-Dade County,Miami,Florida,United States
2,HAMPTON INN STES MIAMI AIRPORT,25.77887,-80.28834,Miami-Dade County,Miami,Florida,United States
3,BEST WESTERN PLUS MIAMI AIRPOR,25.80835,-80.28552,Miami-Dade County,Miami Springs,Florida,United States
4,COMFORT INN AND SUITES MIAMI AIRPORT,25.80799,-80.28107,Miami-Dade County,Miami Springs,Florida,United States


In [94]:
df4 = pd.merge(car, df2, on='name', how='left')

In [95]:
df4.head()

,name,latitude,longitude,County,City_x,State_x,Country,gmap_id,City_y,State_y,url
0,SOFITEL MIAMI,25.78307,-80.29356,Miami-Dade County,Miami,Florida,United States,NaN,NaN,NaN,NaN
1,HILTON MIAMI AIRPORT,25.78241,-80.28058,Miami-Dade County,Miami,Florida,United States,NaN,NaN,NaN,NaN
2,HAMPTON INN STES MIAMI AIRPORT,25.77887,-80.28834,Miami-Dade County,Miami,Florida,United States,NaN,NaN,NaN,NaN
3,BEST WESTERN PLUS MIAMI AIRPOR,25.80835,-80.28552,Miami-Dade County,Miami Springs,Florida,United States,NaN,NaN,NaN,NaN
4,COMFORT INN AND SUITES MIAMI AIRPORT,25.80799,-80.28107,Miami-Dade County,Miami Springs,Florida,United States,NaN,NaN,NaN,NaN


In [110]:
cantida = df1['gmap_id'].nunique()

In [111]:
df1.head()

,text,gmap_id
0,Update: Their “reply” to my review amounted to...,0x8893863ea87bd5dd:0x9383ebf973e74abb
1,He's a knowledgeable doctor but the way he run...,0x8893863ea87bd5dd:0x9383ebf973e74abb
2,"Best doctor I've ever had, I never wait to be ...",0x8893863ea87bd5dd:0x9383ebf973e74abb
3,I was told he is a good doctor. I was trying t...,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,Takes the time to actually get to know his pat...,0x8893863ea87bd5dd:0x9383ebf973e74abb


In [113]:
cantida

129644